# Use frozen pb file to do predict

refers to https://github.com/peter6888/tensorflow/blob/c47c4f09d0da12bec7b184d25ca19f9e0a034e7d/tensorflow/examples/label_image/label_image.py

But Tensorflow example doesn't provide two or more layer inputs.

To-do: 
1. load *.pb model 
2. feed *.pb model with correct inputs
3. read output layer and values

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import tensorflow as tf

In [3]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

def read_tensor_from_image_file(file_name, input_height=299, input_width=299,
				input_mean=0, input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3,
                                       name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                  name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                        name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.Session()
  result = sess.run(normalized)

  return result

def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

In [4]:

root_path = '/home/stonepeter/notebooks/keras-frcnn'
pbfile = 'frozen_model_classifier.pb'
input_layers = ['input_6','input_5'] # refs to https://github.com/peter6888/keras-frcnn/blob/master/test.ipynb
# "input_layers": [["input_6", 0, 0], ["input_5", 0, 0]], "layers": [{"class_name": "InputLayer", "name": "input_6", "inbound_nodes": [], "config": {"sparse": false, "name": "input_6", "dtype": "float32", "batch_input_shape": [null, null, null, 1024]}}, {"class_name": "InputLayer", "name": "input_5", "inbound_nodes": [], "config": {"sparse": false, "name": "input_5", "dtype": "float32", "batch_input_shape": [null, 32, 4]}},